In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# %%
import torch
import string

In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.4 MB/s 


In [4]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 7.4 MB/s 
     |████████████████████████████████| 895 kB 60.6 MB/s 
     |████████████████████████████████| 636 kB 78.1 MB/s 
     |████████████████████████████████| 3.3 MB 52.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [180]:
with open('stockholm/wikipedia/verbs/wikipedia_masked_verbs.txt', 'r') as fp:
    text = fp.read().split('\n')

##BERT

In [8]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
cd drive/My Drive/Colab Notebooks/experiments

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/experiments'
/content/drive/My Drive/Colab Notebooks/experiments


In [10]:
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd
data = pd.read_csv("data/moh-x.csv")

# Split to train, val and test
text, test_df = tts(data[["sentence", "arg1", "verb", "label"]], random_state=42, test_size=0.1)
text, val = tts(text, random_state=42, test_size=test_df.shape[0])

In [11]:
text = text["sentence"]

In [12]:
#text.to_csv(r'mohx.txt', header=None, index=None, sep=' ', mode='a')

In [13]:
with open('data/mohx.txt', 'r') as fp:
    text = fp.read().split('\n')

In [14]:
text[:5]

['" I ca n\'t buy this story ."',
 '" European children learn the breast stroke ; they often do n\'t know how to crawl ."',
 '" He shed his image as a pushy boss ."',
 '" She devoured his novels ."',
 '" Age and experience mellowed him over the years ."']

In [15]:
inputs = tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [16]:
inputs

{'input_ids': tensor([[  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,  2002,  ...,     0,     0,     0],
        ...,
        [  101,  1000, 13862,  ...,     0,     0,     0],
        [  101,  1000,  2016,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [17]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [18]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [19]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [20]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [21]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [22]:
selection[:5]

[[10, 11], [4, 6, 8, 11], [2, 11], [], [2, 9]]

In [23]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [24]:
inputs.input_ids

tensor([[  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,   103,  ...,     0,     0,     0],
        ...,
        [  101,  1000, 13862,  ...,     0,     0,     0],
        [  101,  1000,  2016,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]])

In [25]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [26]:
dataset = MeditationsDataset(inputs)

In [27]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [28]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#model.to(device)

In [29]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [30]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [01:10<00:00,  2.13s/it, loss=0.0684]


##XLM

In [31]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [32]:
inputs = xlmroberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [33]:
inputs

{'input_ids': tensor([[     0,     44,     87,  ...,      1,      1,      1],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,      1,      1,      1],
        ...,
        [     0,     44,  14838,  ...,      1,      1,      1],
        [     0,     44,   4687,  ...,      1,      1,      1],
        [     0,      2,      1,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [34]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [35]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [36]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [37]:
mask_arr

tensor([[False,  True,  True,  ..., False, False,  True],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ...,  True, False,  True],
        ...,
        [False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False,  True, False],
        [False, False, False,  ..., False, False, False]])

In [38]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [39]:
selection[:5]

[[1, 2, 15, 20], [7], [6, 9, 10, 14, 18, 20], [6, 9, 12], [11]]

In [40]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [41]:
inputs.input_ids

tensor([[     0,    103,    103,  ...,      1,      1,    103],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,    103,      1,    103],
        ...,
        [     0,    103,  14838,  ...,      1,      1,      1],
        [     0,     44,   4687,  ...,      1,    103,      1],
        [     0,      2,      1,  ...,      1,      1,      1]])

In [42]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [43]:
dataset = MeditationsDataset(inputs)

In [44]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [45]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#xlmroberta_model.to(device)

In [46]:
from transformers import AdamW

# activate training mode
xlmroberta_model.train()
# initialize optimizer
optim = AdamW(xlmroberta_model.parameters(), lr=5e-5)

In [47]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = xlmroberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [03:08<00:00,  5.72s/it, loss=0.625]


In [48]:
bert_model = model.eval()

In [49]:
xlmroberta_model = xlmroberta_model.eval()

##BART

In [50]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [51]:
inputs = bart_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [52]:
inputs

{'input_ids': tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [53]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [54]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [55]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [56]:
mask_arr

tensor([[False,  True, False,  ...,  True, False, False],
        [False, False, False,  ...,  True, False, False],
        [False, False, False,  ...,  True, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ...,  True, False,  True]])

In [57]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [58]:
selection[:5]

[[1, 5, 16, 18], [6, 18], [5, 9, 11, 16, 18], [9, 11], [3, 12, 13, 15]]

In [59]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [60]:
inputs.input_ids

tensor([[    0,   103,    38,  ...,   103,     1,     1],
        [    0,   113,   796,  ...,   103,     2,     1],
        [    0,   113,    91,  ...,   103,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,   103,     1,   103]])

In [61]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [62]:
dataset = MeditationsDataset(inputs)

In [63]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [64]:
from transformers import AdamW

# activate training mode
bart_model.train()
# initialize optimizer
optim = AdamW(bart_model.parameters(), lr=5e-5)

In [65]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = bart_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [04:09<00:00,  7.57s/it, loss=0.338]


In [66]:
bart_model = bart_model.eval()

##RoBERTa

In [67]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [68]:
inputs = roberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [69]:
inputs

{'input_ids': tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   796,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [70]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [71]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [72]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [73]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False,  True,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [74]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [75]:
selection[:5]

[[8, 10, 11, 16], [2, 11], [8, 9, 14], [9, 17, 19], [14]]

In [76]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [77]:
inputs.input_ids

tensor([[    0,   113,    38,  ...,     1,     1,     1],
        [    0,   113,   103,  ..., 39058,     2,     1],
        [    0,   113,    91,  ...,     1,     1,     1],
        ...,
        [    0,   113, 24841,  ...,     1,     1,     1],
        [    0,   113,   264,  ...,     1,     1,     1],
        [    0,     2,     1,  ...,     1,     1,     1]])

In [78]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [79]:
dataset = MeditationsDataset(inputs)

In [80]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [81]:
from transformers import AdamW

# activate training mode
roberta_model.train()
# initialize optimizer
optim = AdamW(roberta_model.parameters(), lr=5e-5)

In [82]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = roberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 2: 100%|██████████| 33/33 [01:18<00:00,  2.38s/it, loss=2.67]


In [83]:
roberta_model = roberta_model.eval()

##Masked Language Modeling Pipeline

In [84]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [85]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

In [86]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [87]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/662 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

In [88]:
top_k = 10

In [89]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [90]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [91]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

## after bart and roberta fine-tuning

In [92]:
text_sentence = "The wildfire <mask> through the forest at an amazing speed."
get_all_predictions(text_sentence)

The wildfire <mask> through the forest at an amazing speed.


{'bart': 'tore\nripped\nraced\nburned\nswept',
 'bert': 'spread\nripped\nswept\ntore\nflashed',
 'electra': 'runs\npasses\nflows\nsweeps\nswept',
 'roberta': 'tore\nswept\nburned\nmoved\nspread',
 'xlm': 'runs\nspread\npassed\nslid\nrun',
 'xlnet': 'through\nwildfire\nfire\nroaring\ncircles'}

##after bart and roberta fine-tuning

In [93]:
text_sentence = "The window panes were <mask> as the wind blew through them"
get_all_predictions(text_sentence)

The window panes were <mask> as the wind blew through them


{'bart': 'smashed\nshattered\nblown\nrattled\nall',
 'bert': 'smashed\ncracked\nshattered\ndark\nblack',
 'electra': 'clear\nvisible\ncovered\nclosed\nopen',
 'roberta': 'shattered\nmelted\nshattering\ncracked\ndrying',
 'xlm': 'open\nhot\nblue\nclosed\nwide',
 'xlnet': 'as\njust\nwere\nsuch\nopen'}

## after bart and roberta fine-tuning

In [94]:
text_sentence = "To heal and <mask> from death my heart"
get_all_predictions(text_sentence)

To heal and <mask> from death my heart


{'bart': 'release\nshed\nbreathe\nrebuild\nmove',
 'bert': 'keep\nprotect\nsave\npreserve\nguard',
 'electra': 'heal\nrecover\ndie\nlive\nsave',
 'roberta': 'recover\nretreat\nprotect\nretract\nrebuild',
 'xlm': 'recover\nfree\naway\nbreath\nprotect',
 'xlnet': 'and\nfrom\ncome\nright\naway'}

In [95]:
text_sentence ="The tax cut will <mask> the economy"
get_all_predictions(text_sentence)

The tax cut will <mask> the economy


{'bart': 'stimulate\nboost\nfuel\ngrow\nrevive',
 'bert': 'hurt\nboost\nstimulate\nrevive\ndamage',
 'electra': 'hurt\nhelp\nboost\nbenefit\nimprove',
 'roberta': 'stimulate\nboost\nspur\ntransform\nrevive',
 'xlm': 'hurt\naffect\ncut\nboost\nfuel',
 'xlnet': 'all\nwill\nbe\nnot\nthe'}

In [96]:
text_sentence = "Black desert <mask> in iron silences"
get_all_predictions(text_sentence)

Black desert <mask> in iron silences


{'bart': 'sw\ndri\nis\nshines\nro',
 'bert': 'air\nsand\nsoil\nlands\nlies',
 'electra': 'is\nappears\nflies\nes\nfalls',
 'roberta': 'condensed\ndissolved\ncoated\nwrapped\nenclosed',
 'xlm': 's\ned\ncomes\nswimming\nis',
 'xlnet': 'and\nfor\nin\nis\non'}

##Wikipedia

In [181]:
text

['\'\' Music is  <mask> with a vast range of instruments and vocal techniques  <mask> from  <mask> to rapping; there are solely instrumental pieces, solely vocal pieces (such as songs without instrumental accompaniment) and pieces that  <mask> singing and instruments\',\' The word  <mask> from Greek μουσική (mousike; "(art) of the Muses")\',\'In its most general form, the activities  <mask> music as an art form or cultural activity  <mask> the creation of works of music (songs, tunes, symphonies, and so on), the criticism of music, the study of the history of music, and the aesthetic examination of music\',\' Ancient Greek and Indian philosophers  <mask> music in two parts: melodies, as tones  <mask> horizontally, and harmonies as tones  <mask> vertically\',\'\',',
 '\'However, 20th-century composer John Cage  <mask> that any sound  <mask> be music,  <mask> , for example, "There is no noise, only sound\',\' There are many types of music,  <mask> popular music, traditional music, art mu

In [224]:
text_sentence = "Music is performed with a <mask> range of instruments and vocal techniques"
get_all_predictions(text_sentence)

Music is performed with a <mask> range of instruments and vocal techniques


{'bart': 'wide\nrange\ngreat\nfull\nwhole',
 'bert': 'wide\ndiverse\nbroad\nfull\nvaried',
 'electra': 'wide\nbroad\ndiverse\nfull\nwhole',
 'roberta': 'wide\nbroad\nfull\nwider\nbroader',
 'xlm': 'wide\nbroad\ngreat\nlarge\nvast',
 'xlnet': 'a\nrange\nvariety\nwide\ndifferent'}

In [225]:
text_sentence = "The word derives from <mask> μουσική (mousike; (art) of the Muses"
get_all_predictions(text_sentence)

The word derives from <mask> μουσική (mousike; (art) of the Muses


{'bart': 'the\nμ\nGreek\nancient\nAncient',
 'bert': 'greek\nitalian\nlatin\nthe\nsanskrit',
 'electra': 'the\na\n[UNK]\nan\nthis',
 'roberta': 'the\nHercules\nGreek\nβ\n<mask>',
 'xlm': 'the\nhis\na\nold\ntheir',
 'xlnet': 'from\nGreek\n<unk>\nthe\nand'}

In [226]:
text_sentence = "In its most <mask> form, the activities describing music as an art form or cultural activity"
get_all_predictions(text_sentence)

In its most <mask> form, the activities describing music as an art form or cultural activity


{'bart': 'productive\npopular\ncreative\neffective\nadvanced',
 'bert': 'basic\ncomplete\nelaborate\nextreme\nformal',
 'electra': 'recent\ncommon\ntraditional\npopular\noriginal',
 'roberta': 'condensed\nformal\nbasic\nrefined\nnatural',
 'xlm': 'popular\ncurrent\nappropriate\ntraditional\ncreative',
 'xlnet': 'form\nmost\nbasic\nall\nmain'}

In [227]:
text_sentence = "<mask> Greek and Indian philosophers defined music in two parts: melodies, as tones"
get_all_predictions(text_sentence)

<mask> Greek and Indian philosophers defined music in two parts: melodies, as tones


{'bart': '<s>\n</s>\nThe\n�\n,"',
 'bert': 'the\nancient\nclassical\nmedieval\nearly',
 'electra': 'both\nthe\nclassical\nlatin\nmany',
 'roberta': 'The\nTraditional\nthe\nBoth\n<mask>',
 'xlm': 'The\nthe\nBoth\nSome\nThese',
 'xlnet': 'as\nand\nThe\nthe\nAs'}

In [228]:
text_sentence = "However, <mask> composer John Cage thought that any sound can be music"
get_all_predictions(text_sentence)

However, <mask> composer John Cage thought that any sound can be music


{'bart': 'the\ncomposer\nfamed\nrenowned\nacclaimed',
 'bert': 'the\njazz\nclassical\nmodernist\nmusic',
 'electra': 'the\namerican\nformer\nmusic\nclassical',
 'roberta': 'the\nfamed\nfamous\nCage\nmilitary',
 'xlm': 'the\nThe\nhis\ngreat\nour',
 'xlnet': 'the\nand\nwas\nas\na'}

In [229]:
text_sentence = "There are many types of music, including <mask> music, traditional music, art music,"
get_all_predictions(text_sentence)

There are many types of music, including <mask> music, traditional music, art music,


{'bart': 'jazz\nfolk\nclassical\nsym\nblues',
 'bert': 'classical\npopular\nrock\npop\ncontemporary',
 'electra': 'classical\nfolk\nmodern\ntraditional\nelectronic',
 'roberta': 'military\nfolk\nhouse\ncourt\nrock',
 'xlm': 'country\nfolk\nhouse\npop\nelectronic',
 'xlnet': 'popular\npop\nrock\nclassical\nmusic'}

In [230]:
text_sentence = "Music may be played or sung and heard live at a <mask> concert or orchestra performance"
get_all_predictions(text_sentence)

Music may be played or sung and heard live at a <mask> concert or orchestra performance


{'bart': 'concert\nsym\nmusic\nmusical\ntheater',
 'bert': 'classical\nlive\nmusical\nband\nsolo',
 'electra': 'local\nlive\nconcert\nsingle\npublic',
 'roberta': 'live\nsolo\nlarge\nuniversity\nmajor',
 'xlm': 'music\nconcert\nbig\nlarge\ngroup',
 'xlnet': 'concert\nlive\nmusic\nmusical\nprofessional'}

In [231]:
text_sentence = "<mask> activities ranging from amateur karaoke singing to playing in an amateur"
get_all_predictions(text_sentence)

<mask> activities ranging from amateur karaoke singing to playing in an amateur


{'bart': '<s>\n</s>\n�\n,"\nThis',
 'bert': 'recreational\nleisure\noutdoor\nvarious\nadult',
 'electra': 'various\nother\nwith\nthe\nall',
 'roberta': 'sheltered\nutes\n<mask>\n\'"\npan',
 'xlm': 'various\nThe\nmany\nthe\nSome',
 'xlnet': 'ranging\nwhich\nvarying\nas\nstarting'}

In [232]:
text_sentence = "People may make music as a hobby, like a teen playing cello in a youth orchestra, or work as a <mask> musician or singer"
get_all_predictions(text_sentence)

People may make music as a hobby, like a teen playing cello in a youth orchestra, or work as a <mask> musician or singer


{'bart': 'musician\nprofessional\njazz\nsession\ncomposer',
 'bert': 'professional\nsession\nfreelance\njazz\nsolo',
 'electra': 'young\nyouth\nprofessional\nstage\nstudent',
 'roberta': 'studio\nprofessional\ntrained\nsolo\nfreelance',
 'xlm': 'youth\nyoung\nhouse\ncountry\njazz',
 'xlnet': 'professional\na\nworking\ncommercial\nbusiness'}

In [233]:
text_sentence = "The music industry includes the individuals who create new songs and <mask> pieces (such as songwriters and composers)"
get_all_predictions(text_sentence)

The music industry includes the individuals who create new songs and <mask> pieces (such as songwriters and composers)


{'bart': 'musical\nmusic\npieces\noriginal\ncomposing',
 'bert': 'musical\nnew\nsong\nmusic\noriginal',
 'electra': 'new\nother\ncreative\nmusical\ninnovative',
 'roberta': 'art\ndance\ncomposed\nmusic\nnew',
 'xlm': 'music\nnew\nart\ngreat\nmusical',
 'xlnet': 'and\nother\nnew\nmusic\na'}

In [234]:
text_sentence = "<mask> music is art music produced or rooted in the traditions of Western culture"
get_all_predictions(text_sentence)

<mask> music is art music produced or rooted in the traditions of Western culture


{'bart': '<s>\n</s>\n,"\n�\nThe',
 'bert': 'western\nart\nindian\nrenaissance\ncontemporary',
 'electra': 'art\nmusic\nreligious\ncultural\nartistic',
 'roberta': 'Art\nart\nTraditional\nart\nCow',
 'xlm': 'Marathi\nHouse\ncountry\nPop\nModern',
 'xlnet': 'music\nis\nor\nand\npop'}

In [235]:
text_sentence = "The <mask> periods are the medieval (500–1400), Renaissance (1400–1600), Baroque (1600–1750), Classical (1750–1820), Romantic (1800–1910), Modernist (1890–1975) and Postmodern era"
get_all_predictions(text_sentence)

The <mask> periods are the medieval (500–1400), Renaissance (1400–1600), Baroque (1600–1750), Classical (1750–1820), Romantic (1800–1910), Modernist (1890–1975) and Postmodern era


{'bart': 'dominant\nprevailing\nperiods\nmajor\nartistic',
 'bert': 'oldest\nhistorical\nkey\nclassic\nmiddle',
 'electra': 'earliest\nearly\nperiod\nfollowing\ntraditional',
 'roberta': 'key\ntwo\ndominant\nthree\nother',
 'xlm': 'various\nfollowing\nmain\ndifferent\nmajor',
 'xlnet': 'historical\nmodern\nclassical\nClassical\nModern'}

In [236]:
text_sentence = "The <mask> reference to classical music recorded by the Oxford English Dictionary is from about 1829"
get_all_predictions(text_sentence)

The <mask> reference to classical music recorded by the Oxford English Dictionary is from about 1829


{'bart': 'first\nearliest\nonly\noldest\nlast',
 'bert': 'earliest\nfirst\noldest\nlast\nonly',
 'electra': 'earliest\nfirst\nonly\nmain\noriginal',
 'roberta': 'first\nlast\nonly\nearliest\noldest',
 'xlm': 'first\nlast\noriginal\nlatest\nbest',
 'xlnet': 'first\nFirst\nlast\nearliest\nThe'}

In [237]:
text_sentence = "<mask> staff notation is used by composers to indicate to the performer"
get_all_predictions(text_sentence)

<mask> staff notation is used by composers to indicate to the performer


{'bart': '<s>\n</s>\nThe\n�\n,"',
 'bert': 'the\na\nmusical\nthis\nnotation',
 'electra': 'the\na\nthis\nclassical\nstandard',
 'roberta': 'Traditional\nThe\nthe\noral\n<mask>',
 'xlm': 'The\nThis\nthe\na\nthis',
 'xlnet': 'and\nhead\nde\nstaff\nregular'}

In [238]:
text_sentence = "It includes both <mask> (religious) and secular music"
get_all_predictions(text_sentence)

It includes both <mask> (religious) and secular music


{'bart': 'religious\nclassical\nsacred\ntraditional\nChristian',
 'bert': 'religious\nsacred\nsecular\nclassical\nchristian',
 'electra': 'classical\ntraditional\nmusic\nlive\nsecular',
 'roberta': 'oral\nPagan\nfast\nangels\nsacred',
 'xlm': 'political\nalternative\npopular\nmusic\ntraditional',
 'xlnet': 'both\neither\nthe\nin\na'}

In [239]:
text_sentence = "Electronic music is music that employs electronic musical instruments, <mask> instruments"
get_all_predictions(text_sentence)

Electronic music is music that employs electronic musical instruments, <mask> instruments


{'bart': 'keyboards\nsynthes\nincluding\npercussion\noften',
 'bert': 'electronic\nmusical\nclassical\nincluding\nlike',
 'electra': 'electronic\nmusical\nand\nor\nphysical',
 'roberta': 'electrical\nmusical\npercussion\nsound\nvocal',
 'xlm': 'musical\nvarious\nelectronic\nspecial\nmany',
 'xlnet': 'electronic\nand\nother\nmusical\ninstruments'}

In [240]:
text_sentence = "<mask> electronic instruments depended entirely on circuitry-based sound generation"
get_all_predictions(text_sentence)

<mask> electronic instruments depended entirely on circuitry-based sound generation


{'bart': '<s>\n</s>\nThe\n�\n,"',
 'bert': 'early\nmost\nall\nmany\nthe',
 'electra': 'all\nmost\nthe\nwhen\nmany',
 'roberta': 'Traditional\nMost\nMany\nmany\nBoth',
 'xlm': 'The\nThese\nSome\nThis\nthe',
 'xlnet': 'electronic\nElectronic\nand\ninstrument\ne'}

In [241]:
text_sentence = "Electromechanical instruments can have <mask> parts such as strings, hammers, and electric elements"
get_all_predictions(text_sentence)

Electromechanical instruments can have <mask> parts such as strings, hammers, and electric elements


{'bart': 'musical\nmany\ndifferent\nsolid\nother',
 'bert': 'metal\nelectronic\nmechanical\nmultiple\nmetallic',
 'electra': 'other\nvarious\ndifferent\nmechanical\nexternal',
 'roberta': 'flexible\nintegral\nmoving\nmany\ncomponent',
 'xlm': 'many\nvarious\nnumerous\ndifferent\nseveral',
 'xlnet': 'have\nbody\nhaving\nbe\nhad'}

In [242]:
text_sentence = " such <mask> devices include the telharmonium, Hammond organ, electric piano and the electric guitar"
get_all_predictions(text_sentence)

 such <mask> devices include the telharmonium, Hammond organ, electric piano and the electric guitar


{'bart': 'devices\nmusical\nmusic\naudio\nperformance',
 'bert': 'musical\nelectronic\nsound\nmechanical\naudio',
 'electra': 'organ\nmusical\ninstrument\nother\nsuch',
 'roberta': 'electrical\nnew\nkey\nproduction\nsupplementary',
 'xlm': 'other\nvarious\nmany\nthe\nproduction',
 'xlnet': 'such\nas\na\nother\n’'}

In [243]:
text_sentence = "The first electronic <mask> devices were developed at the end of the 19th century"
get_all_predictions(text_sentence)

The first electronic <mask> devices were developed at the end of the 19th century


{'bart': 'devices\nlistening\ncommunication\near\nhearing',
 'bert': 'medical\ncommunication\ncommunications\nelectronic\nlistening',
 'electra': 'electronic\ndevice\nmobile\ncommunication\nmedical',
 'roberta': 'communication\nelectrical\nproduction\ncommunications\nmilitary',
 'xlm': 's\ncontrol\ncommunication\nhealth\ncomputer',
 'xlnet': 'electronic\ndevices\nelectronics\nelectrical\ne'}

In [244]:
text_sentence = "During the 1920s and 1930s, a number of <mask> instruments were introduced "
get_all_predictions(text_sentence)

During the 1920s and 1930s, a number of <mask> instruments were introduced 


{'bart': 'musical\nstring\nnew\npercussion\njazz',
 'bert': 'new\nmusical\nstring\npercussion\nelectronic',
 'electra': 'new\nmusical\nelectronic\nother\nclassical',
 'roberta': 'new\nmusical\npercussion\nelectrical\nmilitary',
 'xlm': 'new\nvarious\nmusical\ndifferent\ngreat',
 'xlnet': 'of\nmore\nmusical\nother\nmajor'}

In [245]:
text_sentence = "By the 1940s, <mask> audio tape allowed musicians to tape sounds"
get_all_predictions(text_sentence)

By the 1940s, <mask> audio tape allowed musicians to tape sounds


{'bart': 'audio\nthe\nmagnetic\na\ncompact',
 'bert': 'the\nan\nelectronic\nthis\nportable',
 'electra': 'the\nelectronic\ndigital\nstandard\nrecorded',
 'roberta': 'the\nan\ndigital\ncompressed\ntransistor',
 'xlm': 'the\nnew\nold\ncustom\ncheap',
 'xlnet': 'an\nwhat\nwhen\nwhere\nthat'}

In [246]:
text_sentence = "Musique concrète, created in Paris in 1948, was based on editing together <mask> fragments of natural and industrial sounds"
get_all_predictions(text_sentence)

Musique concrète, created in Paris in 1948, was based on editing together <mask> fragments of natural and industrial sounds


{'bart': 'fragments\nthe\ndisparate\nmany\nseveral',
 'bert': 'the\nmany\nnumerous\nexisting\nold',
 'electra': 'the\nwith\nvarious\nlarge\ntwo',
 'roberta': 'the\nfragmented\nsmall\ndisparate\nedited',
 'xlm': 'various\nthe\nmany\nsome\nsmall',
 'xlnet': 'fragments\ntogether\napart\nsmall\nlarge'}

In [247]:
text_sentence = "Music produced solely from <mask> generators was first produced in Germany in 1953"
get_all_predictions(text_sentence)

Music produced solely from <mask> generators was first produced in Germany in 1953


{'bart': 'generators\ndiesel\nsteam\ngenerator\nthe',
 'bert': 'steam\nchemical\nnuclear\ndiesel\nelectrical',
 'electra': 'the\npower\nelectric\nmetal\nthese',
 'roberta': 'electrical\npower\nsteam\nplant\nelectricity',
 'xlm': 'oil\nnuclear\npower\nthe\nwind',
 'xlnet': 'generator\nthe\ngenerators\nonly\nno'}

In [248]:
text_sentence = "In the <mask> 1970s, Moog synthesizers and Japanese drum machines helped popularize synthesized electronic music"
get_all_predictions(text_sentence)

In the <mask> 1970s, Moog synthezisers and Japanese drum machines helped popularize synthesized electronic music


{'bart': 'late\nearly\nmid\n1960\n1970',
 'bert': 'early\nlate\nmid\nlater\nlatter',
 'electra': 'early\nlate\nmid\nmiddle\nlater',
 'roberta': 'late\nearly\nmid\nsummer\nlater',
 'xlm': 'late\nearly\nmid\nsummer\ngreat',
 'xlnet': 'the\nsame\n1970\n1980\n’'}

In [249]:
text_sentence = "mass <mask> digital synthesizers, such as the Yamaha DX7, became popular and MIDI"
get_all_predictions(text_sentence)

mass <mask> digital synthesizers, such as the Yamaha DX7, became popular and MIDI


{'bart': 'synthes\ndigital\nbass\nreproduction\ninstruments',
 'bert': 'produced\nmarket\nproduction\nconsumer\nproducing',
 'electra': 'and\nproduced\nelectric\nproduction\nised',
 'roberta': 'plant\naged\nage\nwatered\nproduction',
 'xlm': 'produced\nged\nof\nproduction\nand',
 'xlnet': 'mass\nelectronic\nnew\ncomputer\ndigital'}

In [250]:
text_sentence = "A music video is a short film or video, of <mask> length, that integrates a song or album with imagery"
get_all_predictions(text_sentence)

A music video is a short film or video, of <mask> length, that integrates a song or album with imagery


{'bart': 'a\nabout\nseveral\none\nless',
 'bert': 'varying\nany\nvariable\nsome\nconsiderable',
 'electra': 'varying\nany\nfull\nsome\nvariable',
 'roberta': 'any\nvarying\nequal\nvariable\nunknown',
 'xlm': 'great\nsome\nconsiderable\ngood\nlarge',
 'xlnet': 'of\nlength\nshort\nor\na'}

In [251]:
text_sentence = "Tie-ins and merchandising can be used for toys or for food or <mask> products"
get_all_predictions(text_sentence)

Tie-ins and merchandising can be used for toys or for food or <mask> products


{'bart': 'other\ndrinks\ndrink\nsnack\nhousehold',
 'bert': 'beverage\ndairy\nconsumer\nfood\nchemical',
 'electra': 'other\nfor\nfood\nhousehold\nclothing',
 'roberta': 'other\nhome\ncosmetic\nbaby\nplant',
 'xlm': 'other\nbath\nbeauty\nhealth\nenergy',
 'xlnet': 'or\nother\nany\nproducts\nfood'}

In [252]:
text_sentence = "Although the origins of the music video date back to musical <mask> films that first appeared in the 1920"
get_all_predictions(text_sentence)

Although the origins of the music video date back to musical <mask> films that first appeared in the 1920


{'bart': 'films\nshorts\ncomed\ntheatre\nshort',
 'bert': 'comedy\nshort\ntheatre\ntheater\ncomedies',
 'electra': 'video\ncomedy\naudio\ndrama\nmusic',
 'roberta': 'silent\ntheatre\ncomedy\nhorror\ntheater',
 'xlm': 'comedy\nfantasy\ns\naction\nproduction',
 'xlnet': 'film\ntelevision\nfilms\nvideo\ntheatre'}

In [253]:
text_sentence = "Prior to the 1980s, these kinds of videos were described by <mask> terms"
get_all_predictions(text_sentence)

Prior to the 1980s, these kinds of videos were described by <mask> terms


{'bart': 'computer\ntheir\nHollywood\nprofessional\nacademic',
 'bert': 'the\ngeneric\nthese\nsimilar\ntechnical',
 'electra': 'the\ntwo\nthree\nthese\ndifferent',
 'roberta': 'umbrella\ngeneric\npolite\nformal\nshorthand',
 'xlm': 'various\nmany\ndifferent\nseveral\nother',
 'xlnet': 'by\nin\nwhat\nthe\nway'}

In [254]:
text_sentence = "music videos combine a <mask> range of styles and contemporary video"
get_all_predictions(text_sentence)

music videos combine a <mask> range of styles and contemporary video


{'bart': 'wide\nrange\nwhole\ngreat\ndiverse',
 'bert': 'wide\ndiverse\nbroad\nvast\nlarge',
 'electra': 'wide\nbroad\ndiverse\nwhole\nbroader',
 'roberta': 'wide\nbroad\nfull\ndiverse\nwider',
 'xlm': 'wide\ngreat\nbroad\nlarge\nwhole',
 'xlnet': 'a\nwide\nvaried\nrange\nfull'}

In [255]:
text_sentence = "Some music videos combine <mask> styles with music, such as animation and live-action"
get_all_predictions(text_sentence)

Some music videos combine <mask> styles with music, such as animation and live-action


{'bart': 'styles\ndifferent\nother\nmovie\ndance',
 'bert': 'visual\nanimation\nart\nanimated\nvideo',
 'electra': 'different\nmusic\ntraditional\nother\ntheir',
 'roberta': 'other\nmany\ndifferent\nnew\nart',
 'xlm': 'various\ndifferent\nmany\nthe\nsome',
 'xlnet': 'animation\nartistic\ncreative\nin\ninteractive'}

In [256]:
text_sentence = "other music videos may not have any concept, being only a <mask> version of the song "
get_all_predictions(text_sentence)

other music videos may not have any concept, being only a <mask> version of the song 


{'bart': 'shortened\ncover\nlive\nvideo\nmusical',
 'bert': 'new\nvideo\nshortened\nshort\ncondensed',
 'electra': 'single\nstylized\nlive\nvisual\nshort',
 'roberta': 'condensed\nshortened\nrewritten\ncompressed\ndiluted',
 'xlm': 'small\nrough\nshort\nnew\ntentative',
 'xlnet': 'a\npart\nsmall\nlimited\npartial'}

In [257]:
text_sentence = "Some types of folk music may be called <mask> music"
get_all_predictions(text_sentence)

Some types of folk music may be called <mask> music


{'bart': 'blues\njazz\nsym\nfolk\ntraditional',
 'bert': 'folk\ncountry\nclassical\nbluegrass\npopular',
 'electra': 'folk\nclassical\ntraditional\nurban\ncountry',
 'roberta': 'military\nfolk\nbird\ntraditional\nformal',
 'xlm': 'country\nfolk\nhouse\ngroup\nrap',
 'xlnet': 'called\nas\nfolk\ngathered\nthe'}

In [258]:
text_sentence = "It has been contrasted with <mask> and classical styles"
get_all_predictions(text_sentence)

It has been contrasted with <mask> and classical styles


{'bart': 'the\nother\nclassical\njazz\nopera',
 'bert': 'baroque\nclassical\nrenaissance\nbyzantine\nromanesque',
 'electra': 'modern\nclassical\ncontemporary\nmedieval\ntraditional',
 'roberta': 'military\ntraditional\nEuropean\nmodern\nclean',
 'xlm': 'modern\nromantic\nexotic\nold\ntraditional',
 'xlnet': 'and\nwith\nto\nby\nor'}

In [259]:
text_sentence = "This type of folk music also includes <mask> genres such as folk rock, folk metal, and others"
get_all_predictions(text_sentence)

This type of folk music also includes <mask> genres such as folk rock, folk metal, and others


{'bart': 'other\nmusic\nmany\nstyles\nmusical',
 'bert': 'other\nfolk\nrelated\nnew\nmusical',
 'electra': 'other\nvarious\nsome\ncertain\nmusic',
 'roberta': 'other\nmany\nnew\nnewer\nseveral',
 'xlm': 'various\nmany\nother\nsome\ndifferent',
 'xlnet': 'such\nthose\nthe\nmusical\nother'}

In [260]:
text_sentence = "While contemporary folk music is a genre generally distinct from <mask> folk music"
get_all_predictions(text_sentence)

While contemporary folk music is a genre generally distinct from <mask> folk music


{'bart': 'traditional\nfolk\nclassical\nblues\nCeltic',
 'bert': 'traditional\nclassical\nmodern\nclassic\nold',
 'electra': 'contemporary\nmodern\ntraditional\nwestern\nclassical',
 'roberta': 'traditional\nformal\nold\nolder\nconventional',
 'xlm': 'traditional\nmodern\nold\nthe\npopular',
 'xlnet': 'traditional\na\nconventional\nthe\nbeing'}

In [261]:
text_sentence = "it often shares the <mask> performers and venues as traditional folk music"
get_all_predictions(text_sentence)

it often shares the <mask> performers and venues as traditional folk music


{'bart': 'stage\nmusic\nperformers\nrepertoire\nspace',
 'bert': 'same\nmany\nvarious\nsimilar\nbest',
 'electra': 'same\ntraditional\nmusic\nlocal\nother',
 'roberta': 'same\nlive\nmany\nstage\nnew',
 'xlm': 'same\nvarious\nmany\ndifferent\nbest',
 'xlnet': 'same\nsuch\ncommon\nthe\nmain'}

In [262]:
text_sentence = " music can be divided into genres in <mask> ways, such as popular music and art music, or religious music and secular music"
get_all_predictions(text_sentence)

 music can be divided into genres in <mask> ways, such as popular music and art music, or religious music and secular music


{'bart': 'many\nseveral\ndifferent\nother\ntwo',
 'bert': 'various\ndifferent\nseveral\nmany\nmultiple',
 'electra': 'different\nvarious\nmany\nother\nseveral',
 'roberta': 'many\ndifferent\nseveral\nvarious\nother',
 'xlm': 'various\nmany\nnumerous\ndifferent\nseveral',
 'xlnet': 'some\ncertain\nother\ndifferent\nvarious'}

In [137]:
text_sentence = "R&B chart in the 1960s, and many recordings <mask> over into the pop charts"
get_all_predictions(text_sentence)

R&B chart in the 1960s, and many recordings <mask> over into the pop charts


{'bart': 'spilled\ncrossed\nhave\nflowed\npoured',
 'bert': 'crossed\ncarried\nspilled\ncrossing\ncross',
 'electra': 'came\nspilled\nrolled\nwent\nmoved',
 'roberta': 'crossed\ncross\nspilled\nspill\ntipped',
 'xlm': 'carried\nhead\nspread\nturned\ncame',
 'xlnet': 'over\nout\nwell\nback\ntop'}

In [138]:
text_sentence = "Britain and elsewhere,  <mask> by the Japanese conglomerate Sony Group Corporation, it is part of the Sony Music Group"
get_all_predictions(text_sentence)

Britain and elsewhere,  <mask> by the Japanese conglomerate Sony Group Corporation, it is part of the Sony Music Group


{'bart': 'it\nowned\npartly\nabsorbed\nrun',
 'bert': 'owned\nadministered\nrun\ncontrolled\noperated',
 'electra': 'owned\noperated\nrun\ncontrolled\nfunded',
 'roberta': 'owned\nabsorbed\ncontrolled\nadministered\nmanufactured',
 'xlm': 'run\nled\ndeveloped\nproduced\nmanaged',
 'xlnet': 'and\nowned\ncontrolled\noperated\nproduced'}

In [139]:
text_sentence = "It was originally  <mask> in 1929 as american record corporation"
get_all_predictions(text_sentence)

It was originally  <mask> in 1929 as american record corporation


{'bart': 'incorporated\nfounded\nformed\norganized\nestablished',
 'bert': 'established\nfounded\nformed\nlaunched\nincorporated',
 'electra': 'established\nincorporated\nknown\nfounded\nformed',
 'roberta': 'formed\nconceived\nestablished\ncreated\nfounded',
 'xlm': 'established\ncreated\nbought\nannounced\ndeveloped',
 'xlnet': 'in\nlater\noriginally\nfirst\nreleased'}

In [140]:
text_sentence = "The merger was  <mask> on August 1, 2019"
get_all_predictions(text_sentence)

The merger was  <mask> on August 1, 2019


{'bart': 'finalized\neffective\nofficially\nformally\ncomplete',
 'bert': 'completed\nfinalized\nannounced\napproved\ncomplete',
 'electra': 'announced\nlaunched\nconcluded\ncompleted\nfinalized',
 'roberta': 'finalized\ncompleted\ninitiated\napproved\nsigned',
 'xlm': 'announced\nplanned\nreported\ndecided\napproved',
 'xlnet': 'was\non\nagain\nthen\nmade'}

In [141]:
text_sentence = "Country music often  <mask> of ballads and dance tunes with generally simple forms, folk lyrics, and harmonies "
get_all_predictions(text_sentence)

Country music often  <mask> of ballads and dance tunes with generally simple forms, folk lyrics, and harmonies 


{'bart': 'consists\nsings\nconsist\nincorporates\ncomposed',
 'bert': 'consists\nconsisted\nconsist\nconsisting\ncomposed',
 'electra': 'consists\nspeaks\nconsist\nconsisted\ntalks',
 'roberta': 'consists\ncomprises\nconsisted\ncomposed\nconsist',
 'xlm': 'consist\nmade\nfull\ncompose\nthought',
 'xlnet': 'often\nconsists\nprimarily\nmostly\nconsisting'}

In [142]:
text_sentence = "It  <mask> more than three million album entries and 30 million tracks"
get_all_predictions(text_sentence)

It  <mask> more than three million album entries and 30 million tracks


{'bart': 'sold\nhas\nstreamed\nreleased\nhad',
 'bert': 'accumulated\nhad\ncontained\ncontains\namassed',
 'electra': 'has\nhad\nincluded\nincludes\ncontains',
 'roberta': 'accumulated\namassed\nhad\nhas\nproduced',
 'xlm': 'has\nhad\nis\nproduced\ns',
 'xlnet': 'more\nhas\nis\nmakes\nthem'}

In [143]:
text_sentence = "<mask> in 1991, the database was first made available on the Internet in 1994'"
get_all_predictions(text_sentence)

<mask> in 1991, the database was first made available on the Internet in 1994'


{'bart': '<s>\n</s>\n�\n,"\nThe',
 'bert': 'created\nestablished\ncompleted\nfounded\nbegun',
 'electra': 'and\nbuilt\nback\nfounded\nstarting',
 'roberta': 'Met\nreed\nFed\nspread\ninitiated',
 'xlm': "''\n</s>\nWhether\n«\nMeet",
 'xlnet': 'in\nagain\ncopyright\nrenewed\ned'}

In [144]:
text_sentence = "AllMusic is  <mask> by RhythmOne"
get_all_predictions(text_sentence)

AllMusic is  <mask> by RhythmOne


{'bart': 'powered\nstreamed\npublished\nbacked\nplaying',
 'bert': 'compiled\nreviewed\nprovided\nproduced\nrun',
 'electra': 'produced\ndirected\nwritten\nedited\nreviewed',
 'roberta': 'programmed\ncomposed\nadministered\nsupplied\nproduced',
 'xlm': 'produced\nprovided\noffered\nrun\ndesigned',
 'xlnet': 'is\nwas\nmade\nprovided\ndone'}

In [145]:
text_sentence = "Music! Music! Music! is a popular song  <mask> by Stephen Weiss and Bernie Baum "
get_all_predictions(text_sentence)

Music! Music! Music! is a popular song  <mask> by Stephen Weiss and Bernie Baum 


{'bart': 'written\ncomposed\nsung\nrecorded\nperformed',
 'bert': 'written\ncomposed\nrecorded\npenned\nsung',
 'electra': 'written\ncomposed\nsung\nperformed\ninspired',
 'roberta': 'composed\nwritten\ncreated\ndeveloped\nconceived',
 'xlm': 'written\nsung\nproduced\ninspired\ncreated',
 'xlnet': 'by\ndirected\nservice\nstarted\nplan'}

In [146]:
text_sentence = "EDM producers also  <mask> their music live in a concert"
get_all_predictions(text_sentence)

EDM producers also  <mask> their music live in a concert


{'bart': 'play\nplayed\nperformed\nperform\ndecided',
 'bert': 'perform\nplay\nperformed\nperforms\nbroadcast',
 'electra': 'play\nrecorded\nput\nperform\nplayed',
 'roberta': 'play\nperform\nsell\npresent\ndeliver',
 'xlm': 'make\nput\nplay\nbring\nplayed',
 'xlnet': 'also\ntheir\nmade\nmake\nrecorded'}

In [147]:
text_sentence = "UMG\'s corporate headquarters are <mask> in Hilversum, Netherlands"
get_all_predictions(text_sentence)

UMG's corporate headquarters are <mask> in Hilversum, Netherlands


{'bart': 'located\nsituated\nbased\npermanently\nnow',
 'bert': 'located\nbased\nsituated\nheadquartered\nalso',
 'electra': 'located\nbased\nheadquartered\nsituated\ncentered',
 'roberta': 'located\nsituated\nbased\nspread\nalso',
 'xlm': 'located\nestablished\nbuilt\nnow\nbased',
 'xlnet': 'in\nare\nat\nlocated\nbased'}

In [148]:
text_sentence = "UMG has  <mask> licensing agreements with more than 400 platforms worldwide"
get_all_predictions(text_sentence)

UMG has  <mask> licensing agreements with more than 400 platforms worldwide


{'bart': 'signed\nseveral\nentered\nmultiple\nexclusive',
 'bert': 'signed\nhad\nreciprocal\nlicensed\nbilateral',
 'electra': 'signed\ndeveloped\nreached\nestablished\nhad',
 'roberta': 'signed\nnegotiated\nsecured\nexclusive\nforged',
 'xlm': 'signed\ndeveloped\nreached\nmade\nbuilt',
 'xlnet': 'has\nhad\nhave\ninternational\nglobal'}

In [149]:
text_sentence = "Gospel music is a genre of Christian music, Gospel music can be <mask> to the early 17th century"
get_all_predictions(text_sentence)

Gospel music is a genre of Christian music, Gospel music can be <mask> to the early 17th century


{'bart': 'traced\nattributed\ndated\ntrace\nasc',
 'bert': 'traced\ndated\ndates\naccredited\nattributed',
 'electra': 'traced\ndated\nattributed\nused\nadapted',
 'roberta': 'dated\ntraced\nattributed\nextended\ndating',
 'xlm': 'related\nback\ntied\ncompared\nassociated',
 'xlnet': 'be\nback\nfound\ncan\ndated'}

In [150]:
text_sentence = "Most of the churches  <mask> on hand clapping and foot stomping as rhythmic accompaniment"
get_all_predictions(text_sentence)

Most of the churches  <mask> on hand clapping and foot stomping as rhythmic accompaniment


{'bart': 'rely\nrelied\ndepend\nleaned\nare',
 'bert': 'relied\nfocused\nconcentrated\nsettled\nrely',
 'electra': 'rely\nfocus\ndepend\nrelied\ninsist',
 'roberta': 'relied\ninsisted\ndepended\nrely\ninsist',
 'xlm': 'take\nwork\ndepend\nare\nturn',
 'xlnet': 'are\non\nup\nchurches\nis'}

In [151]:
text_sentence = "Most of the singing was <mask> a cappella "
get_all_predictions(text_sentence)

Most of the singing was <mask> a cappella 


{'bart': 'done\nin\nfrom\naccompanied\na',
 'bert': 'sung\nalmost\nmostly\ndone\nperformed',
 'electra': 'in\non\nmade\njust\nfrom',
 'roberta': 'in\nsung\npitched\ndelivered\ncomposed',
 'xlm': 'in\nby\nwith\nthrough\nfrom',
 'xlnet': 'a\nin\nan\nwas\njust'}

In [152]:
text_sentence = "The first  <mask> use of the term gospel song probably  <mask> in 1874"
get_all_predictions(text_sentence)

The first  <mask> use of the term gospel song probably  <mask> in 1874


{'bart': 'conscious\nrecorded\nformal\nuse\nwidespread',
 'bert': 'recorded\nwritten\ndocumented\nknown\nsystematic',
 'electra': 'known\npopular\ncommon\nrecorded\npublic',
 'roberta': 'formal\nrecorded\nwidespread\nproper\nsustained',
 'xlm': 'official\nknown\npopular\ncommercial\nsuccessful',
 'xlnet': 'hit\ntime\nsight\never\nin'}

In [153]:
text_sentence = "The original gospel songs were <mask> and composed by authors such as George F"
get_all_predictions(text_sentence)

The original gospel songs were <mask> and composed by authors such as George F


{'bart': 'written\nsung\narranged\ncomposed\nall',
 'bert': 'written\ntranscribed\ncomposed\nsung\narranged',
 'electra': 'written\ncomposed\nrecorded\nsung\nproduced',
 'roberta': 'written\nsung\ncomposed\nconceived\nresearched',
 'xlm': 'written\ndeveloped\ncreated\nplanned\ndesigned',
 'xlnet': 'and\nwritten\nmade\ncomposed\nwas'}

In [154]:
text_sentence = "Southern gospel  <mask> all male, tenor-lead-baritone-bass quartet"
get_all_predictions(text_sentence)

Southern gospel  <mask> all male, tenor-lead-baritone-bass quartet


{'bart': 'is\nsingers\nmusic\nsings\nsinging',
 'bert': 'choir\nquartet\nmusic\nchorus\nband',
 'electra': 'includes\nhas\nis\nthe\nprovides',
 'roberta': 'comprises\nfused\nfeatures\nis\nencompasses',
 'xlm': 'is\nwas\nhas\nare\nwith',
 'xlnet': 'all\ngospel\nchurch\nAll\nthe'}

In [155]:
text_sentence = "British black gospel  <mask> to Gospel music of the African diaspora"
get_all_predictions(text_sentence)

British black gospel  <mask> to Gospel music of the African diaspora


{'bart': 'music\nrefers\nmusicians\nowes\nanswers',
 'bert': 'refers\napplied\nreferred\nrefer\nreferring',
 'electra': 'music\nrefers\ncontributed\nconnections\ncontributions',
 'roberta': 'fused\nspread\nadded\ndanced\ncontributed',
 'xlm': 's\nlinks\ncompared\nrelated\naccording',
 'xlnet': 'to\ngospel\nand\nthe\nwith'}

In [156]:
text_sentence = "Into the Music is the 11th studio album by Northern Irish singer-songwriter Van Morrison, and was  <mask> in August 1979"
get_all_predictions(text_sentence)

Into the Music is the 11th studio album by Northern Irish singer-songwriter Van Morrison, and was  <mask> in August 1979


{'bart': 'released\nrecorded\nissued\noriginally\nofficially',
 'bert': 'released\nrecorded\nissued\ncompleted\npublished',
 'electra': 'released\nrecorded\npublished\nannounced\nreissued',
 'roberta': 'released\nrecorded\ncompleted\nissued\npressed',
 'xlm': 'released\npublished\nproduced\nwritten\ncompleted',
 'xlnet': 'released\nmade\nissued\npublished\non'}

In [157]:
text_sentence = "The Music are an English alternative rock band,  <mask> in Kippax, Leeds in 1999"
get_all_predictions(text_sentence)

The Music are an English alternative rock band,  <mask> in Kippax, Leeds in 1999


{'bart': 'formed\nwho\nfounded\nbased\nhaving',
 'bert': 'formed\nforming\nfounded\nbased\nestablished',
 'electra': 'formed\nfounded\nestablished\nreleased\nstarted',
 'roberta': 'formed\nforming\nfounded\nborn\nestablished',
 'xlm': 'established\nborn\nopened\nbased\ncreated',
 'xlnet': 'performed\nmade\nappeared\ndescribed\nand'}

In [158]:
text_sentence = "YouTube Music is a music streaming service  <mask> by YouTube, a subsidiary of Google"
get_all_predictions(text_sentence)

YouTube Music is a music streaming service  <mask> by YouTube, a subsidiary of Google


{'bart': 'operated\nowned\nrun\noffered\nlaunched',
 'bert': 'operated\nprovided\noffered\nowned\ncreated',
 'electra': 'provided\noffered\nowned\noperated\nrun',
 'roberta': 'owned\ncreated\nlaunched\ndeveloped\ndistributed',
 'xlm': 'offered\ndeveloped\nrun\nprovided\nproduced',
 'xlnet': 'provided\nservice\noffered\nmade\nserved'}

In [159]:
text_sentence = "It  <mask> a  tailored interface for the service"
get_all_predictions(text_sentence)

It  <mask> a  tailored interface for the service


{'bart': 'created\nis\nhas\ndesigned\nused',
 'bert': 'provided\nwas\noffered\nhad\ncreated',
 'electra': 'provides\nprovided\nis\nwas\nhas',
 'roberta': 'created\nwas\nformed\ngave\nwanted',
 'xlm': 'has\nprovides\nis\noffers\ngives',
 'xlnet': 'it\nis\nIt\na\nin'}

In [160]:
text_sentence = "The service also  <mask> a premium tier"
get_all_predictions(text_sentence)

The service also  <mask> a premium tier


{'bart': 'offered\noffers\nhas\nsubscribed\ncharges',
 'bert': 'offers\nhad\nfeatures\nadded\noffered',
 'electra': 'has\noffers\nfeatures\nincludes\nprovides',
 'roberta': 'has\nfeatures\noffers\nadded\nhad',
 'xlm': 'has\noffers\nprovides\ndeveloped\nreceived',
 'xlnet': 'also\nas\nstill\nwith\non'}

In [161]:
text_sentence = "These subscription benefits are also <mask> to subscribers of Google Play Music and YouTube Premium"
get_all_predictions(text_sentence)

These subscription benefits are also <mask> to subscribers of Google Play Music and YouTube Premium


{'bart': 'available\naccessible\nextended\nbeneficial\ncomplimentary',
 'bert': 'available\noffered\nextended\nprovided\ngiven',
 'electra': 'available\nprovided\naccessible\nextended\ngiven',
 'roberta': 'extended\navailable\ngranted\nafforded\nexpanded',
 'xlm': 'offered\navailable\nattractive\nopen\ngiven',
 'xlnet': 'to\nnot\nonly\nalso\navailable'}

In [162]:
text_sentence = "In the 1960s, several British rock bands such as the Rolling Stones, the Who and the Animals were  <mask> to and promoted as being R&B bands"
get_all_predictions(text_sentence)

In the 1960s, several British rock bands such as the Rolling Stones, the Who and the Animals were  <mask> to and promoted as being R&B bands


{'bart': 'referred\nlistened\nadmitted\noften\nmistakenly',
 'bert': 'referred\ncompared\nlikened\nlistened\nintroduced',
 'electra': 'referred\nlinked\npromoted\nsigned\nopened',
 'roberta': 'referred\nlistened\nadded\nchanged\nattached',
 'xlm': 'married\nbrought\nreported\nled\nsold',
 'xlnet': 'referred\nalso\nto\ninitially\nall'}

In [163]:
text_sentence = "It  <mask> guest stars such as Ella Fitzgerald, John Davidson and Nelson Riddle and his orchestra"
get_all_predictions(text_sentence)

It  <mask> guest stars such as Ella Fitzgerald, John Davidson and Nelson Riddle and his orchestra


{'bart': 'starred\nfeatured\nwelcomed\nplayed\nused',
 'bert': 'featured\nincluded\nfeatures\nstarred\nhad',
 'electra': 'features\nhas\nincludes\nfeatured\nstars',
 'roberta': 'drew\nfeatures\nattracted\ndraws\nattracts',
 'xlm': 'has\nhad\nfeatures\nincludes\ns',
 'xlnet': 'guest\nis\nin\nit\nhas'}

In [164]:
text_sentence = "Music, Music is the eighth studio album by Australian-American pop singer Helen Reddy that was <mask> in the summer of 1976 by Capitol Records"
get_all_predictions(text_sentence)

Music, Music is the eighth studio album by Australian-American pop singer Helen Reddy that was <mask> in the summer of 1976 by Capitol Records


{'bart': 'released\nissued\nrecorded\nrelease\npublished',
 'bert': 'released\nissued\nrecorded\nproduced\ndistributed',
 'electra': 'released\nreissued\nrecorded\npublished\nsold',
 'roberta': 'released\nissued\npressed\nrecorded\ndistributed',
 'xlm': 'released\nproduced\nannounced\npublished\nsigned',
 'xlnet': 'released\nproduced\nrecorded\nmade\nissued'}

In [165]:
text_sentence = "It  <mask> on Billboard\\'s Top LP\\'s & Tapes chart that same month"
get_all_predictions(text_sentence)

It  <mask> on Billboard\'s Top LP\'s & Tapes chart that same month


{'bart': 'debuted\nchart\ntopped\npeaked\nclimbed',
 'bert': 'appeared\ndebuted\nlanded\nwas\ncharted',
 'electra': 'appeared\ndebuted\nwas\npremiered\nwent',
 'roberta': 'landed\ndebuted\nappeared\narrived\nwent',
 'xlm': 'appeared\nwas\nappears\nis\ncame',
 'xlnet': '”\nOn\nIt\nchart\nThere'}

In [166]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'released\nhailed\nacclaimed\nchosen\naccepted',
 'bert': 'released\nselected\nrecorded\ndescribed\nissued',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'conceived\nadopted\nchosen\nselected\nnamed',
 'xlm': 'written\nproduced\ncreated\nsung\ndiscovered',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [167]:
text_sentence = "On August 23, 2005, Music, Music was  <mask> for the first time on compact disc as one of two Helen Reddy albums on one CD"
get_all_predictions(text_sentence)

On August 23, 2005, Music, Music was  <mask> for the first time on compact disc as one of two Helen Reddy albums on one CD


{'bart': 'released\npressed\nrecorded\nissued\nplayed',
 'bert': 'reissued\nreleased\nissued\nrecorded\nfeatured',
 'electra': 'released\nrecorded\nreissued\npublished\nremastered',
 'roberta': 'released\npressed\ncondensed\nplayed\npresented',
 'xlm': 'written\npublished\nproduced\nused\nreleased',
 'xlnet': 'released\npresented\nannounced\navailable\nalso'}

In [168]:
text_sentence = "Music Choice is  <mask> nationwide by  \u202fComcast Xfinity,\u202fCharter Spectrum,\u202fCox Communications,\u202fVerizon Fios,\u202fDirecTV\u202fand many other mid-size and smaller  MVPDs"
get_all_predictions(text_sentence)

Music Choice is  <mask> nationwide by   Comcast Xfinity, Charter Spectrum, Cox Communications, Verizon Fios, DirecTV and many other mid-size and smaller  MVPDs


{'bart': 'carried\nstreamed\ndistributed\nbeing\nnow',
 'bert': 'offered\nprovided\ndistributed\nbroadcast\nhandled',
 'electra': 'accessed\noffered\navailable\ndistributed\nprovided',
 'roberta': 'administered\nspread\nshared\ncarried\nexpanded',
 'xlm': 'offered\nrun\ndeveloped\nmanaged\nreported',
 'xlnet': 'provided\nalso\ns\nis\nmade'}

In [169]:
text_sentence = "music is a top-level domain name (TLD) <mask> in 2003 for use in music dissemination and appreciation"
get_all_predictions(text_sentence)

music is a top-level domain name (TLD) <mask> in 2003 for use in music dissemination and appreciation


{'bart': 'that\nregistered\ncreated\nlicensed\nlaunched',
 'bert': 'created\nestablished\nformed\nlaunched\nfounded',
 'electra': 'established\ncreated\ndeveloped\nintroduced\nadopted',
 'roberta': 'created\nregistered\nestablished\nformed\npatented',
 'xlm': 'established\ncreated\ndeveloped\nregistered\nbuilt',
 'xlnet': 'created\nused\nnamed\nlaunched\nmade'}

In [170]:
text_sentence = "Limited  <mask> that their rights application had been approved by the Internet Corporation for Assigned Names and Numbers (ICANN)"
get_all_predictions(text_sentence)

Limited  <mask> that their rights application had been approved by the Internet Corporation for Assigned Names and Numbers (ICANN)


{'bart': 'liability\ncompanies\nbusiness\ncorporations\nparties',
 'bert': 'now\nknowledge\nrecognition\nevidence\nnotice',
 'electra': 'notice\nstated\ninformation\nclaims\nnote',
 'roberta': 'proof\nSpectrum\nutes\ncountered\nmeaning',
 'xlm': 'thought\nbelief\ndecided\nbelieve\nrealized',
 'xlnet': 'Limited\nand\nsaid\nstated\nin'}

In [171]:
text_sentence = "K-pop (Korean: 케이팝; RR: keipap), short for Korean popular music, is a genre of music <mask> in South Korea as part of South Korean culture"
get_all_predictions(text_sentence)

K-pop (Korean: 케이팝; RR: keipap), short for Korean popular music, is a genre of music <mask> in South Korea as part of South Korean culture


{'bart': 'originating\nthat\npopular\nplayed\nplaying',
 'bert': 'popular\noriginating\nproduced\ncreated\noriginated',
 'electra': 'recorded\nproduced\npopular\nused\nplayed',
 'roberta': 'developed\nadopted\ntaught\npracticed\npopular',
 'xlm': 'developed\nproduced\nmade\ncreated\nliving',
 'xlnet': 'popular\nplayed\nperformed\nused\nin'}

In [172]:
text_sentence = "The term K-pop  <mask> popular in the 2000s"
get_all_predictions(text_sentence)

The term K-pop  <mask> popular in the 2000s


{'bart': 'became\nis\nwas\ngrew\ngot',
 'bert': 'became\nwas\nremained\ngrew\nbecome',
 'electra': 'became\nwas\ngot\ngrew\nproved',
 'roberta': 'became\nwas\ngrew\nwent\ngot',
 'xlm': 'became\nwas\nis\ngot\nproved',
 'xlnet': 'pop\npopular\npop\na\nmore'}

In [173]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'released\nhailed\nacclaimed\nchosen\naccepted',
 'bert': 'released\nselected\nrecorded\ndescribed\nissued',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'conceived\nadopted\nchosen\nselected\nnamed',
 'xlm': 'written\nproduced\ncreated\nsung\ndiscovered',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [174]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'released\nhailed\nacclaimed\nchosen\naccepted',
 'bert': 'released\nselected\nrecorded\ndescribed\nissued',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'conceived\nadopted\nchosen\nselected\nnamed',
 'xlm': 'written\nproduced\ncreated\nsung\ndiscovered',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [175]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'released\nhailed\nacclaimed\nchosen\naccepted',
 'bert': 'released\nselected\nrecorded\ndescribed\nissued',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'conceived\nadopted\nchosen\nselected\nnamed',
 'xlm': 'written\nproduced\ncreated\nsung\ndiscovered',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [176]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'released\nhailed\nacclaimed\nchosen\naccepted',
 'bert': 'released\nselected\nrecorded\ndescribed\nissued',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'conceived\nadopted\nchosen\nselected\nnamed',
 'xlm': 'written\nproduced\ncreated\nsung\ndiscovered',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}

In [177]:
text_sentence = "The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums"
get_all_predictions(text_sentence)

The third album recorded by Reddy with producer Joe Wissert, Music, Music was  <mask> in 1977 by Reddy as a personal favorite from among her albums


{'bart': 'released\nhailed\nacclaimed\nchosen\naccepted',
 'bert': 'released\nselected\nrecorded\ndescribed\nissued',
 'electra': 'released\nrecorded\npublished\nannounced\ncredited',
 'roberta': 'conceived\nadopted\nchosen\nselected\nnamed',
 'xlm': 'written\nproduced\ncreated\nsung\ndiscovered',
 'xlnet': 'released\nnamed\nlisted\nselected\nmarketed'}